In [18]:
import numpy as np
import pandas as pd

df = pd.read_excel('/Users/elena/Downloads/data-4209-2019-10-18.xlsx')

In [19]:
col = df.columns

In [69]:
def find_costs(s): #Функция ищет стоимость посещения коворкинга в ячейке
    costs = []
    s = str(s)
    s = s.lower().replace('-', ' ').replace(';', ' ').replace('кв.м.', ' ').replace(',', '.')
    for item in s.replace('-', ' ').split():
        if ('.' in item and item[:item.find('.')].isdigit()) or (item.isdigit()):
            costs.append(item)
    return pd.Series(list(map(float, costs)))

In [71]:
tariffs = []
for i in range(df.shape[0]):
    tariffs.append(find_costs(df['Tariffs'][i]).mean()) #Найдем средний тариф коворкинга. Игнорируем другие чиста
    print(find_costs(df['Tariffs'][i]))

0      800.0
1     4000.0
2    14900.0
3    16900.0
dtype: float64
0      800.0
1     4000.0
2    14900.0
3    16900.0
dtype: float64
0      150.0
1      400.0
2     2000.0
3     7900.0
4     5900.0
5    11900.0
dtype: float64
0      500.0
1    12500.0
2      500.0
3      650.0
dtype: float64
0     3500.0
1    25000.0
2    50000.0
3     8000.0
4     3125.0
5    90000.0
dtype: float64
0    10132.563156
dtype: float64
0        1.0
1      800.0
2    11000.0
3    14000.0
4    16000.0
5     1500.0
dtype: float64
0        1.0
1    17000.0
2        3.0
3    16000.0
4    15500.0
5     1000.0
dtype: float64
0    100.0
1     36.0
2     30.0
3     11.0
4    300.0
5     22.0
6    500.0
dtype: float64
0    0.0
dtype: float64
0       3.0
1    5000.0
2    7000.0
dtype: float64
0    250000.0
1    200000.0
2     15000.0
3      3500.0
4      2000.0
5      7000.0
6      4000.0
dtype: float64
0       0.99
1    1190.00
dtype: float64
0       0.99
1    1190.00
dtype: float64
0    10132.563156
dtype: float64

In [64]:
df['Tariffs'] = df['Tariffs'].fillna(pd.Series(tariffs).mean())

In [80]:
is_free = []
t = list(df['Tariffs'])
for item in list(df['Tariffs']):
    s = str(item)
    s = s.lower().replace('(', ' ').replace(')', ' ')
    is_free.append(int('-0.00' in str(item) or 'бесплант' in str(item)))

In [81]:
df['is_free'] = is_free

In [82]:
print('Доля коворкингов, где хотя бы одна услуга бесплатная, составляет', df['is_free'].value_counts(normalize=True)[1])

Доля коворкингов, где хотя бы одна услуга бесплатная, составляет 0.012195121951219513


In [83]:
print('Количество коворкингов с хотя бы одной бесплатной услугой', df['is_free'].value_counts()[1])

Количество коворкингов с хотя бы одной бесплатной услугой 2


In [76]:
max_cost = []
for i in range(df.shape[0]):
    if len(find_costs(df['Tariffs'][i])) == 0:
        max_cost.append(0.0)
    else:
        max_cost.append(max(find_costs(df['Tariffs'][i])))
df['max_cost'] = max_cost

In [77]:
print(df[df['max_cost'] == max(max_cost)]['FullName'], ' - коворкинг, в котором можно отдать максимальную сумму денег единовременно')

11    Коворкинг «Meeting Point»
Name: FullName, dtype: object  - коворкинг, в котором можно отдать максимальную сумму денег единовременно


In [78]:
import requests
from bs4 import BeautifulSoup
import urllib 

In [ ]:
titles = []
q_t = []
q_ref = []
for i in range(df.shape[0]):
    try:
        if '/' in df['Site'][i]:
            response = requests.get('http://'+df['Site'][i])
        else: 
            response = requests.get('http://'+df['Site'][i]+'/')
        soup = BeautifulSoup(response.text, 'html.parser')
        l = []
        for item in soup.findAll('a'):
            l.append(item.get('href'))
        links = []
        for item in l:
            if 'http' in str(item) and not(df['Site'][i] in str(item)):
                links.append(item)
        title = str(soup.title)
        titles.append(title[title.find('>')+1:title.rfind('<')])
        q_t.append(soup.get_text().count(str(df['FullName'][i])))
        q_ref.append(len(links))
    except:
        titles.append('None')
        q_t.append('nan')
        q_ref.append('nan')
        

In [100]:
df['title_html'] = titles
df['кол-во title'] = q_t
df['кол-во ссылок'] = q_ref

In [101]:
df

,ID,FullName,global_id,ShortName,ShortDescription,AdmArea,District,Address,LocationClarification,Tariffs,...,Specialization,ContactPhone,Email,Site,geoData,is_free,max_cost,title_html,кол-во title,кол-во ссылок
0,25330,Коворкинг «Рабочая станция» Парк Горького,861999679,Коворкинг «Рабочая станция» Парк Горького,"Рабочая Станция — офис, на поиск и ремонт кото...",Южный административный округ,Донской район,"город Москва, Ленинский проспект, дом 30",Ленинский проспект (558 м.),Тариф Гость (1 день)-800.00 Незакрепленное раб...,...,NaN,(495) 150-01-17,hello@coworkstation.ru,www.coworkstation.ru/,"{type=Point, coordinates=[37.583455975827, 55....",0,16900.000000,e>Круглосуточный коворкинг-центр Рабочая Станц...,0,70
1,25331,Коворкинг «Рабочая станция» ARTPLAY,861999681,Коворкинг «Рабочая станция» ARTPLAY,"Рабочая Станция — офис, на поиск и ремонт кото...",Центральный административный округ,Басманный район,"город Москва, Нижняя Сыромятническая улица, до...",этаж 7,"Тариф ""Гость"" (день)-800.00 Незакрепленное раб...",...,NaN,(495) 150-01-17,hello@coworkstation.ru,coworkstation.ru,"{type=Point, coordinates=[37.671012249348344, ...",0,16900.000000,e>Круглосуточный коворкинг-центр Рабочая Станц...,0,6
2,25332,Коворкинг «Свободное плавание»,861999695,Коворкинг «Свободное плавание»,Коворкинг «Свободное плавание» - это организов...,Южный административный округ,Нагорный район,"город Москва, Варшавское шоссе, дом 28А",NaN,"Рабочее место (Тарифный план ""Час"")-150.00 Нез...",...,NaN,(499) 426-06-26,coworking_nagatino@mail.ru,freecoworking.ru,"{type=Point, coordinates=[37.61798356024735, 5...",0,11900.000000,e>Коворкинг «Свободное плавание» | Аренда рабо...,1,3
3,25333,Коворкинг «Lerner»,861999696,Коворкинг «Lerner»,"Предоставление рабочих мест, переговорных комн...",Южный административный округ,Даниловский район,"город Москва, Павловская улица, дом 18",NaN,Незакрепленное рабочее место (Тариф «Рабочий д...,...,NaN,(499) 951-07-28,Coworking@ler-ner.ru,www.kovorkingi.ru/kovorking/lerner,"{type=Point, coordinates=[37.62630315095879, 5...",0,12500.000000,e>`Пространство развития `ЛЕРНЕР`: тарифы ково...,0,85
4,25335,Коворкинг центр «Cabinet Lounge»,861999697,Коворкинг центр «Cabinet Lounge»,Клубный офис Cabinet Lounge — ваш представител...,Центральный административный округ,Тверской район,"город Москва, Малый Черкасский переулок, дом 2",NaN,Незакрепленное рабочее место (тариф FLEX)-3500...,...,IТ,(495) 790-74-70,vs@cabinetiounge.com,cabinetlounge.com/coworking,"{type=Point, coordinates=[37.62606115629907, 5...",0,90000.000000,e>COWORKING</title>,0,21
5,25340,Коворкинг «DI Telegraph»,861999698,Коворкинг «DI Telegraph»,Telegraph by Rent24 – уникальное гибридное про...,Центральный административный округ,Тверской район,"город Москва, Тверская улица, дом 7",подъезд 9,10132.6,...,IТ,(495) 580-35-21,contact@ditelegraph.ru,ditelegraph.ru/,"{type=Point, coordinates=[37.611068237888695, ...",0,10132.563156,None,nan,nan
6,25341,Клубный офис и коворкинг «Деловар»,861999699,Клубный офис и коворкинг «Деловар»,Клуб предпринимателей Деловар работает с 2003 ...,Центральный административный округ,район Хамовники,"город Москва, Комсомольский проспект, дом 28","здание МДМ, 4 этаж",Рабочее место (на 1 день)-800.00 Незакрепленно...,...,NaN,(495) 177-01-23,delovar-cw@yandex.ru,coworking-delovar.ru/,"{type=Point, coordinates=[37.57970774751588, 5...",0,16000.000000,e></title>,1,13
7,25345,Коворкинг «Arma Coworking»,861999701,Коворкинг «Arma Coworking»,АРМА — уникальная территория бывшего газового ...,Центральный административный округ,Басманный район,"город Москва, Нижний Сусальный переулок, дом 5...",NaN,Закрепленное рабочее место ( 1 месяц)-17000.00...,...,NaN,(495) 755-42-24,office@armacoworking.ru,armacoworking.ru,"{type=Point, coordinates=[37.66469744770028, 5...",0,17000.000000,e>Главная - Arma Coworking</title>,0,1
8,25346,Коворкинг-студия «Столик на одного»,861999702,Коворкинг-студия «Столик на одного»,Столик на одного – это уютный коворкинг в цент...,Центральны